In [5]:
# Original de profesor Miguel Carrasco, UAI, REcon. de patrones de imágenes, Clase 7
# Ligeramente editado por Diego Sandoval R. para la entrega de la Tarea 4

from skimage.feature import greycomatrix, greycoprops
import cv2
import matplotlib.pyplot as plt
import numpy as np 
from sklearn import preprocessing
from math import radians
from skimage.io import imread_collection
import pandas as pd

#cargamos las imágenes de la carpeta indicada
col = imread_collection('texturas/*.bmp')


#submuestreo de 0->14 (15 niveles)
new_scale = (0, 14)
level = 15 #numero de niveles de la imagen 
lista_descriptores = ['contrast', 'dissimilarity','homogeneity', 'ASM', 'energy', 'correlation']


F = []
ntrpM = []

#recorremos cada archivo
for filename in col.files:

    #leemos cada archivo
    image = cv2.imread(filename, 0)

    #submuestreamos cada imagen
    imagen_2 = preprocessing.MinMaxScaler(new_scale).fit_transform(image).astype('int')

    mrg = np.histogramdd(np.ravel(image), bins = 256)[0]/image.size
    mrg = list(filter(lambda p: p > 0, np.ravel(mrg)))
    #entropia
    ntrp = -np.sum(np.multiply(mrg, np.log(mrg)))
    
    ntrpM.append(ntrp)

    #calculamos la matriz de co-ocurrencia (Haralick)
    P = greycomatrix(imagen_2, distances=[1], angles=[radians(0)], levels=level, symmetric=False, normed=False)
    
    #creamos una matriz para guardar los datos
    features = []

    for descriptor in lista_descriptores:
        res = np.squeeze(greycoprops(P,descriptor))
        #agregamos cada descriptor a una lista
        features.append(res.round(5))
        #print(f'{descriptor}: {res:.4f}')
    
    huMomento = cv2.HuMoments(cv2.moments(image))
    nHu = 5

    hu_hedaders = [f'hu{i+1}' for i in range(nHu)]
    for i in range(nHu):
        huMomento[i] = -1* np.copysign(1.0, huMomento[i]) * np.log10(np.abs(huMomento[i]))
        features.append(huMomento[i].round(5)[0])

    F.append(features)

#agregamos los datos a un dataframe
# >> las columnas son los descriptores,
# >> las filas son los nombres de los archivos
df = pd.DataFrame(F, columns=lista_descriptores+hu_hedaders, index=col.files)
df = df.assign(entropy=ntrpM)

print(df)

# Sanity check
#print(ntrpM)

                    contrast  dissimilarity  homogeneity      ASM   energy  \
texturas/00001.bmp   7.06933        2.04325      0.37748  0.01283  0.11328   
texturas/00002.bmp   1.57831        0.90213      0.61304  0.02339  0.15294   
texturas/00003.bmp   1.68079        0.95368      0.59287  0.02322  0.15239   
texturas/00004.bmp   1.79023        0.96937      0.59288  0.01934  0.13907   
texturas/00005.bmp   1.86805        0.98628      0.59006  0.02199  0.14830   
texturas/00006.bmp   3.24668        1.34855      0.49467  0.01721  0.13119   
texturas/00007.bmp   3.07874        1.29478      0.51121  0.01827  0.13515   
texturas/00008.bmp   3.76753        1.44371      0.47905  0.01651  0.12849   
texturas/00009.bmp   3.98997        1.51458      0.46016  0.01682  0.12970   
texturas/00010.bmp   5.69630        1.79694      0.41758  0.01333  0.11546   
texturas/00011.bmp   5.09313        1.69279      0.43338  0.01479  0.12163   
texturas/00012.bmp   4.86251        1.62100      0.45319  0.0184

/home/diego/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
/home/diego/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/diego/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/diego/.local/lib/python3.10/site-packages/skimage/feature/__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
/home/diego/.l